In [4]:
import numpy as np
import itertools as it

In [5]:
#pip install import_ipynb

In [6]:
import import_ipynb
import arnoldi

importing Jupyter notebook from arnoldi.ipynb


In [7]:
from numpy.linalg import inv

In [8]:
import matplotlib.pyplot as plt

In [9]:
#kronecker-delta function

def k_delta(n,m) :
    if n == m :
        return 1
    else :
        return 0

In [10]:
# define states 
def state(mN, pN) : # num of states of mRNA, num of states of protein
    N = mN*pN # num of states

    # states for mRNA and protein
    s = []
    for i in range(mN) :
        for j in range(pN) :
            s.append(np.array([i,j]))
    s = np.array(s)

    return s, N


In [11]:
# stochastic matrix
def stochastic_matrix(mN, pN, avg_m, avg_p, alpha =5, delta = 1) :
    
    # production coefficient of mRNA
    alpha = alpha
    # production coefficient of protein
    betta = avg_p/avg_m
    # degradation coefficient of mRNA
    gamma = alpha/avg_m
    # degradation coefficient of protein
    delta = delta # unit value

    M = np.zeros((N,N))

    Im = np.eye(mN+1)
    Ip = np.eye(pN+1)

    # state
    s, N = state(mN, pN)

    for i in range(N) :
        for j in range(N) :
        
            M[i,j] = alpha*(Im[s[i,0],s[j,0]]*Ip[s[i,1],s[j,1]] - Im[s[i,0]-1,s[j,0]]*Ip[s[i,1],s[j,1]])
            M[i,j] = M[i,j] + betta*s[j,0]*(Im[s[i,0],s[j,0]]*Ip[s[i,1],s[j,1]] - Im[s[i,0],s[j,0]]*Ip[s[i,1]-1,s[j,1]])
            M[i,j] = M[i,j] + gamma*(s[i,0]*Im[s[i,0],s[j,0]]*Ip[s[i,1],s[j,1]] - s[j,0]*Im[s[i,0]+1,s[j,0]]*Ip[s[i,1],s[j,1]])
            M[i,j] = M[i,j] + delta*(s[i,1]*Im[s[i,0],s[j,0]]*Ip[s[i,1],s[j,1]] - s[j,1]*Im[s[i,0],s[j,0]]*Ip[s[i,1]+1,s[j,1]])

    #normalization
    for i in range(N) :
        M[i,i] = 0
        M[i,i] = -np.sum(M[:,i])

    return M
    


In [12]:
def inv_matrix(M) :
    try :
        return inv(M)
    
    except :
        print(f'{M} is singular.')
        return M

In [13]:
# find state at t
def state_t(state_i, mN, pN,t , stochastic_M) :

    N = mN * pN


    # find smallest eigenvalue 
    inv_M = inv_matrix(stochastic_M)
    e_val, e_vec = arnoldi.QR_algorithm(inv_M,100)
    e_val = 1/e_val #eigenvalue of original matrix

    # transit to eigenvector space coordinate
    state_ei = inv(e_vec).dot(state_i)

    # final state
    state_f = 0
    for i in range(N) :
        state_f += state_ei[i]*e_vec[i]*np.exp(-e_val[i]*t)

    sol = np.zeros((mN, pN))
    for i in range(mN) :
        for j in range(pN) :
            sol[i,j] = state_f[mN*i+j]/np.sum(state_f)

    return sol

    

In [17]:
# num of states of mRNA
mN = 10
# num of states of protein
pN = 10
#num of states
N = mN*pN

In [30]:
# states for mRNA and protein
s = []
for i in range(mN) :
    for j in range(pN) :
        s.append(np.array([i,j]))

s = np.array(s)


In [31]:
# average of mRNA
avg_m = 5
# average of protein 
avg_p = 12

In [32]:
# production coefficient of mRNA
alpha = 5
# production coefficient of protein
betta = avg_p/avg_m
# degradation coefficient of mRNA
gamma = alpha/avg_m
# degradation coefficient of protein
delta = 1 # unit value

In [33]:
print(alpha,betta,gamma,delta)

5 2.4 1.0 1


In [34]:
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [35]:
# # Stochastic matrix
# M = np.zeros((N,N))

# I = np.eye()

# for i in range(N) :
#     for j in range(N) :
#         # Diagonal comonent
#         if i == j :
#             M[i, j] = alpha + s[i,0]*betta + s[i,0]*gamma + s[i,1]*delta
        
#         # off-diagonal component

#         # degradation of mRNA(m+1 -> m)
#         elif  s[i,0] - s[j,0] == -1 :
#             if abs(s[i,1] - s[j,1]) != 0 :
#                 break
#             M[i,j] = M[i,j] - s[j,0]*gamma
#         # production of mRNA(m-1 =>m)
#         elif s[i,0] + s[j,0] == 1 :
#             if abs(s[i,1] - s[j,1]) != 0 :
#                 break
#             M[i,j] = M[i,j] - alpha
#         # degradation of protein(p+1 -> p)
#         elif  s[i,1] - s[j,1] == -1 :
#             if abs(s[i,0] - s[j,0]) != 0 :
#                 break
#             M[i,j] = M[i,j] - s[j,1]*delta
#         # degradation of protein(p-1 -> p)
#         elif  s[i,1] - s[j,1] == 1 :
#             if abs(s[i,0] - s[j,0]) != 0 :
#                 break
#             M[i,j] = M[i,j] - s[j,0]*betta



In [36]:
# Stochastic matrix
M = np.zeros((N,N))

Im = np.eye(mN+1)
Ip = np.eye(pN+1)

for i in range(N) :
    for j in range(N) :
        
        M[i,j] = alpha*(Im[s[i,0],s[j,0]]*Ip[s[i,1],s[j,1]] - Im[s[i,0]-1,s[j,0]]*Ip[s[i,1],s[j,1]])
        M[i,j] = M[i,j] + betta*s[j,0]*(Im[s[i,0],s[j,0]]*Ip[s[i,1],s[j,1]] - Im[s[i,0],s[j,0]]*Ip[s[i,1]-1,s[j,1]])
        M[i,j] = M[i,j] + gamma*(s[i,0]*Im[s[i,0],s[j,0]]*Ip[s[i,1],s[j,1]] - s[j,0]*Im[s[i,0]+1,s[j,0]]*Ip[s[i,1],s[j,1]])
        M[i,j] = M[i,j] + delta*(s[i,1]*Im[s[i,0],s[j,0]]*Ip[s[i,1],s[j,1]] - s[j,1]*Im[s[i,0],s[j,0]]*Ip[s[i,1]+1,s[j,1]])

#normalization
for i in range(N) :
    M[i,i] = 0
    M[i,i] = -np.sum(M[:,i])

In [37]:
M[:,3]

array([ 0.,  0., -3.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
       -5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [38]:
np.sum(M, axis = 0)

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -1.77635684e-15,  0.00000000e+00,
        8.88178420e-16,  8.88178420e-16,  8.88178420e-16,  8.88178420e-16,
        8.88178420e-16,  8.88178420e-16,  8.88178420e-16,  8.88178420e-16,
        8.88178420e-16,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        1.77635684e-15,  1.77635684e-15,  1.77635684e-15,  1.77635684e-15,
        1.77635684e-15,  1.77635684e-15,  1.77635684e-15,  1.77635684e-15,
        1.77635684e-15,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [39]:
M

array([[  5. ,  -1. ,   0. , ...,   0. ,   0. ,   0. ],
       [  0. ,   6. ,  -2. , ...,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   7. , ...,   0. ,   0. ,   0. ],
       ...,
       [  0. ,   0. ,   0. , ...,  37.6,  -8. ,   0. ],
       [  0. ,   0. ,   0. , ..., -21.6,  38.6,  -9. ],
       [  0. ,   0. ,   0. , ...,   0. , -21.6,  18. ]])

In [42]:
e_val, e_vec = arnoldi.QR_algorithm(M, 100)

In [43]:
e_val

array([ 6.78459210e+01+0.j        ,  6.03455702e+01+0.j        ,
        5.93584712e+01+0.j        ,  5.44266924e+01+0.j        ,
        5.27291862e+01+0.24678261j,  5.27291862e+01-0.24678261j,
        4.92098366e+01+0.j        ,  4.83507849e+01+0.j        ,
        4.65060928e+01+0.67144806j,  4.65060928e+01-0.67144806j,
        4.44826214e+01+0.j        ,  4.36213842e+01+0.j        ,
        4.18287127e+01+0.j        ,  4.12718969e+01+0.88763961j,
        4.12718969e+01-0.88763961j,  4.01019088e+01+0.j        ,
        3.85152235e+01+0.j        ,  3.72830105e+01+0.20264841j,
        3.72830105e+01-0.20264841j,  3.64356039e+01+1.07968549j,
        3.64356039e+01-1.07968549j,  3.59731976e+01+0.j        ,
        3.30099645e+01+0.25011046j,  3.30099645e+01-0.25011046j,
        3.28545442e+01+0.71390155j,  3.28545442e+01-0.71390155j,
        3.20173128e+01+1.24702333j,  3.20173128e+01-1.24702333j,
        3.20080513e+01+0.j        ,  2.89746669e+01+0.j        ,
        2.82183577e+01+1.

In [44]:
e_vec

array([[ 9.99999982e-01+0.j,  7.63478380e-01+0.j, -4.15221221e-01+0.j,
        ...,  1.52695696e-03+0.j, -9.00909831e-02+0.j,
        -3.31505020e-02+0.j],
       [ 1.87157409e-04+0.j,  6.22964943e-01+0.j, -5.20297145e-01+0.j,
        ...,  1.42615542e-03+0.j,  6.15170742e-02+0.j,
        -9.16690590e-03+0.j],
       [ 7.14662763e-06+0.j,  1.70337095e-01+0.j, -7.46232078e-01+0.j,
        ..., -2.26247420e-03+0.j, -1.18623235e-01+0.j,
        -5.01523632e-02+0.j],
       ...,
       [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        ...,  3.64324933e-01+0.j, -8.63968856e-03+0.j,
         2.45246173e-02+0.j],
       [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        ...,  0.00000000e+00+0.j,  7.46345591e-01+0.j,
        -2.40115416e-01+0.j],
       [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        ...,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         6.65088975e-01+0.j]])

In [45]:
arnoldi.QR_algorithm(M,200)

(array([ 6.78459210e+01+0.j        ,  6.03455702e+01+0.j        ,
         5.93584712e+01+0.j        ,  5.44266924e+01+0.j        ,
         5.27291862e+01+0.24678261j,  5.27291862e+01-0.24678261j,
         4.92098366e+01+0.j        ,  4.83507849e+01+0.j        ,
         4.65060928e+01+0.67144806j,  4.65060928e+01-0.67144806j,
         4.44826214e+01+0.j        ,  4.36213842e+01+0.j        ,
         4.18287127e+01+0.j        ,  4.12718969e+01+0.88763961j,
         4.12718969e+01-0.88763961j,  4.01019088e+01+0.j        ,
         3.85152235e+01+0.j        ,  3.72830105e+01+0.20264841j,
         3.72830105e+01-0.20264841j,  3.64356039e+01+1.07968549j,
         3.64356039e+01-1.07968549j,  3.59731976e+01+0.j        ,
         3.30099645e+01+0.25011046j,  3.30099645e+01-0.25011046j,
         3.28545442e+01+0.71390155j,  3.28545442e+01-0.71390155j,
         3.20173128e+01+1.24702333j,  3.20173128e+01-1.24702333j,
         3.20080513e+01+0.j        ,  2.89746669e+01+0.j        ,
         2

In [46]:
#https://github.com/sh0408sh/NSFCS/tree/be2ebc9a19401de8e3b298c21db9eb91c486ebd1/eigenvalue_prob

In [47]:
# Find the smallest eigenvalue : consider inverse matrix
from numpy.linalg import inv

def inv_matrix(M) :
    try :
        return inv(M)
    
    except :
        print(f'{M} is singular.')
        return M

In [48]:
inv_M = inv_matrix(M)
#eigenvalue of inverse matrix
e_val, e_vec = arnoldi.QR_algorithm(inv_M,100)
e_val = 1/e_val


In [49]:
e_val

array([-3.64203276e-16 -0.j        ,  1.15760275e+00 +0.j        ,
        2.09489984e+00 -0.26120575j,  2.09489984e+00 +0.26120575j,
        3.48961052e+00 +0.j        ,  3.61139747e+00 -1.6863307j ,
        3.61139747e+00 +1.6863307j ,  4.69155429e+00 +0.j        ,
        4.81641874e+00 -0.60342502j,  4.81641874e+00 +0.60342502j,
        5.75546655e+00 -1.64331296j,  5.75546655e+00 +1.64331296j,
        6.52179507e+00 -1.05678878j,  6.52179507e+00 +1.05678878j,
        7.00289233e+00 +0.j        , -1.22990243e+01 -0.j        ,
        7.57522448e+00 -2.51686108j,  7.57522448e+00 +2.51686108j,
        2.93227682e+00-12.21235289j,  2.93227682e+00+12.21235289j,
        8.44197625e+00 -1.10979629j,  8.44197625e+00 +1.10979629j,
        8.67663782e+00 +0.j        ,  9.01104480e+00 -5.29482827j,
        9.01104480e+00 +5.29482827j, -2.10240475e+01 -0.j        ,
        9.67737035e+00 -3.22379738j,  9.67737035e+00 +3.22379738j,
        9.85935792e+00 -1.65735423j,  9.85935792e+00 +1.657354

In [18]:
# initial vector 
basis = np.eye(N)

state_i = basis[1]

In [20]:
state_i.shape

(100,)

In [53]:
#time 
t = 0

In [51]:
# transit to eigenvector space coordinate
state_ei = inv(e_vec).dot(state_i)


In [54]:
# final state
state_f = 0
for i in range(N) :
    state_f += state_ei[i]*e_vec[i]*np.exp(-e_val[i]*t)


In [55]:
sol = np.zeros((mN, pN))
for i in range(mN) :
    for j in range(pN) :
        sol[i,j] = state_f[mN*i+j]/np.sum(state_f)

    

C:\Users\LG\AppData\Local\Temp\ipykernel_8728\2929480359.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  sol[i,j] = state_f[mN*i+j]/np.sum(state_f)


In [57]:
import matplotlib.pyplot as plt

In [21]:
plt.title(f"t= {t}")
plt.xlabel("num of protein")
plt.ylabel("num of mRNA")
plt.imshow(sol, origin="lower")
plt.colorbar()

NameError: name 't' is not defined

In [1]:
from matplotlib.animation import FuncAnimation
from matplotlib.animation import FFMpegWriter

In [ ]:
fig = plt.figure
basis = np.eye(N)
state_i = basis[1]

def animation( t ) :
    fig.clear
